In [1]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
load_dotenv()

True

In [3]:
DB_CONFIG ={
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_NAME'),
    'username': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

In [4]:
try:
    connection_string = (
        f"mysql+pymysql://{DB_CONFIG['username']}:{DB_CONFIG['password']}"
        f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    )   
    engine = create_engine(connection_string)

    with engine.connect() as conn:
        print("conectado ao mysql")

        result = conn.execute(text("SELECT NOW() AS hora_atual"))
        hora = result.fetchone()
        print(f"hora do servidor: {hora[0]}")
except Exception as e:
    print(f"Erro {e}")

conectado ao mysql
hora do servidor: 2025-11-23 15:55:51


In [5]:
query = "SELECT * FROM customers"

In [6]:
df_customers = pd.read_sql(query, engine)
df_customers

,customer_id,nome,age,city
0,1,Bruno Silva,21,Los Angeles
1,2,Caio Jorge,25,San Diego
2,3,Camila Vitória,18,Los Angeles
3,4,Vicente Marques,71,Los Angeles
4,5,Guilherme Freitas,25,New York
5,6,Carlos Magno,48,New York


In [7]:
query2 = " SELECT * FROM orders_details;"

In [8]:
df_order = pd.read_sql(query2, engine)
df_order

,nome,city,product_name,category,quantity,price,order_date,total_value
0,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
1,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
2,Caio Jorge,San Diego,Apple Juice,produce,30,2.75,2025-11-15,82.5
3,Bruno Silva,Los Angeles,Apple Juice,produce,50,2.75,2025-11-09,137.5
4,Carlos Magno,New York,shampoo,personal care,100,3.99,2025-11-12,399.0
5,Camila Vitória,Los Angeles,Past,pantry,20,1.99,2025-11-01,39.8
6,Camila Vitória,Los Angeles,Apple Juice,produce,10,2.75,2025-11-04,27.5
7,Guilherme Freitas,New York,Soda,produce,10,1.00,2025-11-19,10.0
8,Carlos Magno,New York,milk,dairy,100,1.99,2025-11-21,199.0


In [26]:
vendas_por_cliente = df_order.groupby("nome")["total_value"].sum().reset_index()
vendas_por_cliente= vendas_por_cliente.sort_values(by='total_value', ascending=False, ignore_index= True)


In [10]:
vendas_por_produto = df_order.groupby("product_name")["total_value"].sum().reset_index()
vendas_por_produto

,product_name,total_value
0,Apple Juice,247.5
1,Past,39.8
2,Soda,10.0
3,ketchup,599.0
4,milk,199.0
5,shampoo,399.0


In [11]:
quantidade_por_categoria = df_order.groupby("category")["quantity"].sum().reset_index()
quantidade_por_categoria

,category,quantity
0,dairy,100
1,pantry,120
2,personal care,100
3,produce,100


In [23]:
vendas_por_dia = df_order.groupby("order_date")["total_value"].sum().reset_index()
vendas_por_dia.sort_values(by='total_value', ascending=False)

,order_date,total_value
5,2025-11-16,599.0
3,2025-11-12,399.0
7,2025-11-21,199.0
2,2025-11-09,137.5
4,2025-11-15,82.5
0,2025-11-01,39.8
1,2025-11-04,27.5
6,2025-11-19,10.0


# Análise Exploratória dos Dados

In [13]:
df_order.describe()

,quantity,price,total_value
count,9.000000,9.000000,9.000000
mean,46.666667,3.244444,166.033333
std,34.278273,1.751793,140.646578
min,10.000000,1.000000,10.000000
25%,20.000000,1.990000,39.800000
50%,50.000000,2.750000,137.500000
75%,50.000000,3.990000,299.500000
max,100.000000,5.990000,399.000000


In [14]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   nome          9 non-null      object 
 1   city          9 non-null      object 
 2   product_name  9 non-null      object 
 3   category      9 non-null      object 
 4   quantity      9 non-null      int64  
 5   price         9 non-null      float64
 6   order_date    9 non-null      object 
 7   total_value   9 non-null      float64
dtypes: float64(2), int64(1), object(5)
memory usage: 708.0+ bytes


In [15]:
df_order.isnull().sum()

nome            0
city            0
product_name    0
category        0
quantity        0
price           0
order_date      0
total_value     0
dtype: int64

In [28]:
vendas_por_cliente.head(3)

,nome,total_value
0,Bruno Silva,736.5
1,Carlos Magno,598.0
2,Caio Jorge,82.5


In [33]:
vendas_por_produto = vendas_por_produto.sort_values(by='total_value', ascending= False, ignore_index= True)
vendas_por_produto.head(3)

,product_name,total_value
0,ketchup,599.0
1,shampoo,399.0
2,Apple Juice,247.5


In [35]:
quantidade_por_categoria= quantidade_por_categoria.sort_values(by='quantity', ascending= False, ignore_index= True)
quantidade_por_categoria

,category,quantity
0,pantry,120
1,dairy,100
2,personal care,100
3,produce,100


In [36]:
df_order

,nome,city,product_name,category,quantity,price,order_date,total_value
0,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
1,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
2,Caio Jorge,San Diego,Apple Juice,produce,30,2.75,2025-11-15,82.5
3,Bruno Silva,Los Angeles,Apple Juice,produce,50,2.75,2025-11-09,137.5
4,Carlos Magno,New York,shampoo,personal care,100,3.99,2025-11-12,399.0
5,Camila Vitória,Los Angeles,Past,pantry,20,1.99,2025-11-01,39.8
6,Camila Vitória,Los Angeles,Apple Juice,produce,10,2.75,2025-11-04,27.5
7,Guilherme Freitas,New York,Soda,produce,10,1.00,2025-11-19,10.0
8,Carlos Magno,New York,milk,dairy,100,1.99,2025-11-21,199.0


In [45]:
vendas_por_cliente

,nome,total_value
0,Bruno Silva,736.5
1,Carlos Magno,598.0
2,Caio Jorge,82.5
3,Camila Vitória,67.3
4,Guilherme Freitas,10.0


In [48]:
pedidos_por_cliente=df_order.groupby("nome")['quantity'].count().reset_index()
pedidos_por_cliente

,nome,quantity
0,Bruno Silva,3
1,Caio Jorge,1
2,Camila Vitória,2
3,Carlos Magno,2
4,Guilherme Freitas,1


In [49]:
df_ticket = pd.merge(vendas_por_cliente, pedidos_por_cliente, on= 'nome')
df_ticket

,nome,total_value,quantity
0,Bruno Silva,736.5,3
1,Carlos Magno,598.0,2
2,Caio Jorge,82.5,1
3,Camila Vitória,67.3,2
4,Guilherme Freitas,10.0,1


In [50]:
df_ticket['medium ticket'] = df_ticket['total_value']/df_ticket['quantity']
df_ticket

,nome,total_value,quantity,medium ticket
0,Bruno Silva,736.5,3,245.50
1,Carlos Magno,598.0,2,299.00
2,Caio Jorge,82.5,1,82.50
3,Camila Vitória,67.3,2,33.65
4,Guilherme Freitas,10.0,1,10.00


In [53]:
df_order['order_date'] = pd.to_datetime(df_order['order_date'])
df_order

,nome,city,product_name,category,quantity,price,order_date,total_value
0,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
1,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
2,Caio Jorge,San Diego,Apple Juice,produce,30,2.75,2025-11-15,82.5
3,Bruno Silva,Los Angeles,Apple Juice,produce,50,2.75,2025-11-09,137.5
4,Carlos Magno,New York,shampoo,personal care,100,3.99,2025-11-12,399.0
5,Camila Vitória,Los Angeles,Past,pantry,20,1.99,2025-11-01,39.8
6,Camila Vitória,Los Angeles,Apple Juice,produce,10,2.75,2025-11-04,27.5
7,Guilherme Freitas,New York,Soda,produce,10,1.00,2025-11-19,10.0
8,Carlos Magno,New York,milk,dairy,100,1.99,2025-11-21,199.0


In [ ]:
df_order['weekday'] = df_order['order_date'].dt.day_name()

In [56]:
df_order

,nome,city,product_name,category,quantity,price,order_date,total_value,weekday
0,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5,Sunday
1,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5,Sunday
2,Caio Jorge,San Diego,Apple Juice,produce,30,2.75,2025-11-15,82.5,Saturday
3,Bruno Silva,Los Angeles,Apple Juice,produce,50,2.75,2025-11-09,137.5,Sunday
4,Carlos Magno,New York,shampoo,personal care,100,3.99,2025-11-12,399.0,Wednesday
5,Camila Vitória,Los Angeles,Past,pantry,20,1.99,2025-11-01,39.8,Saturday
6,Camila Vitória,Los Angeles,Apple Juice,produce,10,2.75,2025-11-04,27.5,Tuesday
7,Guilherme Freitas,New York,Soda,produce,10,1.00,2025-11-19,10.0,Wednesday
8,Carlos Magno,New York,milk,dairy,100,1.99,2025-11-21,199.0,Friday


In [59]:
vendas_por_dia_semana = df_order.groupby('weekday')['total_value'].sum().reset_index()
vendas_por_dia_semana=vendas_por_dia_semana.sort_values(by='total_value', ascending= False, ignore_index= True)
vendas_por_dia_semana

,weekday,total_value
0,Sunday,736.5
1,Wednesday,409.0
2,Friday,199.0
3,Saturday,122.3
4,Tuesday,27.5


# Vizualização dos Dados